In [12]:
# 質問：2回目
response = chat_engine.stream_chat("交通費以外の手当にはどのようなものがありますか？")

for token in response.response_gen:
    print(token, end="")

2026-02-19 14:24:55,035 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-19 14:24:55,048 - INFO - Condensed question: 交通費以外にどのような手当がありますか？
2026-02-19 14:24:55,808 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-19 14:24:56,778 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


交通費以外の手当には以下のものがあります：

1. **住宅手当**: 通勤に1時間以上かかる場合、月額10,000円が支給されます。住宅手当を受けるためには、賃貸契約書など居住地を証明できる書類の提出が必要です。

2. **家族手当**: 扶養家族がいる従業員には家族手当が支給されます。配偶者には月額5,000円、子供1人につき月額3,000円が支給され、上限は子供3人までです。

3. **時間外手当（残業代）**: 所定の勤務時間を超えて働いた場合に支給されます。残業手当の割増率は法令に基づき計算され、通常の時間外労働は1.25倍、深夜時間帯（午後10時以降）の残業は1.5倍となります。

これらの手当は、従業員の生活をサポートするために設けられています。

In [14]:
# 引用元を表示
for source in response.sources:
    for raw_output in source.raw_output:
        print("ファイル名：", raw_output.node.metadata["file_name"])
        print("関連度スコア:", raw_output.score)
        print("テキスト：")
        print(raw_output.node.text)
        print("-" * 50)  # 区切り線

ファイル名： 02賃金規則.pdf
関連度スコア: 0.8381601805772079
テキスト：
通勤⼿当
通勤にかかる交通費は、実際の経路に基づき⽀給されます。
公共交通機関の利⽤の場合は、最安経路をもとに⽉額上限 3 万円まで⽀給します。
⾃家⽤⾞での通勤が必要な場合は、事前に⼈事部へ申請してください。駐⾞場の使
⽤料やガソリン代の⼀部が⽀給される場合もあります。
2.
--------------------------------------------------
ファイル名： 02賃金規則.pdf
関連度スコア: 0.812901859858618
テキスト：
住宅⼿当
会社から通勤に 1 時間以上かかる場合、住宅⼿当として⽉額 1 万円が⽀給されます。
住宅⼿当を受けるためには、賃貸契約書など、居住地を証明できる書類の提出が必
要です。
3. 家族⼿当
扶養家族がいる従業員には、家族⼿当が⽀給されます。
配偶者には⽉額 5,000 円、⼦供⼀⼈につき⽉額 3,000 円が⽀給されます（上限︓⼦供 3
⼈まで）。
4.
--------------------------------------------------
ファイル名： 02賃金規則.pdf
関連度スコア: 0.7933024800934017
テキスト：
時間外⼿当（残業代）
所定の勤務時間を超えて働いた場合は、残業⼿当が⽀給されます。
残業⼿当の割増率は、法令に基づき計算されます。通常の時間外労働は 1.25 倍、深
夜時間帯（午後 10 時以降）の残業は 1.5 倍となります。
5. 休⽇出勤⼿当
--------------------------------------------------


In [10]:
# 質問：1回目
response = chat_engine.stream_chat("公共交通機関の交通費の上限は？")

for token in response.response_gen:
    print(token, end="")

2026-02-19 14:15:50,735 - INFO - Condensed question: 公共交通機関の交通費の上限は？
2026-02-19 14:15:50,956 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-19 14:15:51,812 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


公共交通機関の利用の場合、交通費は最安経路をもとに月額上限30,000円まで支給されます。

In [9]:
# Chat Engineの作成
chat_engine = index.as_chat_engine(
    chat_mode="best",
    llm=llm,
    similarity_top_k=3,
    text_qa_template=text_qa_template,
    refine_template=refine_template,
)

In [7]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# テキストQAテンプレートの作成
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=sys_prompt_str),
    ChatMessage(
        role=MessageRole.USER,
        content=qa_prompt_str),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# リファインテンプレートの作成
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=sys_prompt_str),
    ChatMessage(
        role=MessageRole.USER,
        content=refine_prompt_str),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

In [6]:
sys_prompt_str = """
事前知識ではなく、常に提供されたコンテキスト情報を使用して質問に回答してください。
回答内でコンテキストを直接参照しないでください。
「コンテキストに基づいて」や「コンテキスト情報は」、またはそれに類するような記述は避けてください。
"""

qa_prompt_str = """
コンテキスト情報は以下の通りです。
---------------------
{context_str}
---------------------
事前知識ではなくコンテキスト情報を使用して、質問に回答してください。
質問: {query_str}
回答："""

refine_prompt_str = """
元の回答を (必要な場合のみ) 以下のコンテキストで改良する機会があります。
-----------
{context_msg}
-----------
新しいコンテキストが与えられた場合、元の回答を改良して、質問 {query_str} に適切に回答します。
コンテキストが役に立たない場合は、元の回答を再度出力します。
元の回答: {existing_answer}"""

In [5]:
from llama_index.core import StorageContext, load_index_from_storage

# ストレージコンテキストの作成
storage_context = StorageContext.from_defaults(persist_dir="./storage01")
# Indexのロード
index = load_index_from_storage(storage_context)

2026-02-19 14:04:25,050 - INFO - Loading all indices.


In [4]:
# ストレージに保存
index.storage_context.persist("./storage01")

In [2]:
# PDFドキュメントの読込
documents = SimpleDirectoryReader('./data/pdf').load_data()

In [3]:
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core import Settings
import tiktoken

# NodeParserの作成
node_parser = SentenceSplitter(
    separator="。",
    chunk_size=256,
    chunk_overlap=16,
    tokenizer=tiktoken.encoding_for_model(MODEL_NAME).encode)

# 言語モデルの指定
llm = OpenAI(model=MODEL_NAME, temperature=0.3)

# 設定に反映
Settings.llm = llm
Settings.node_parser = node_parser

# Indexの構築
index = VectorStoreIndex.from_documents(documents)

2026-02-19 14:02:33,450 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")
os.environ['OPENAI_API_KEY']  = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"